# Synth. Obs.: 3D Phantom - Reduction

We create synthetic observations for the Magritte model of the 3D Phantom snapshot that was created in the [this example](../1_post-processing/4_reduce_Phantom_3D.ipynb).

## Setup

Import the required functionalty.

In [1]:
import magritte.core     as magritte   # Core functionality
import magritte.plot     as plot       # Plotting
import magritte.tools    as tools      # Save fits
import os

from astropy import units              # Unit conversions

Define a working directory (you will have to change this). We assume here that the scripts of the [this example](../1_post-processing/4_reduce_Phantom_3D.ipynb) have already been executed and go back to that working directory.

In [2]:
wdir = "/lhome/thomasc/Magritte-examples/Phantom_3D/"

Define file names.

In [3]:
model_file = os.path.join(wdir, 'model_Phantom_3D_red.hdf5')   # reduced 3D Phantom Magritte model

Load the Magritte model.

In [4]:
model = magritte.Model(model_file)

                                           
-------------------------------------------
  Reading Model...                         
-------------------------------------------
 model file = /lhome/thomasc/Magritte-examples/Phantom_3D/model_Phantom_3D_red.hdf5
-------------------------------------------
Reading parameters...
Reading points...
Reading rays...
Reading boundary...
Reading chemistry...
Reading species...
Reading thermodynamics...
Reading temperature...
Reading turbulence...
Reading lines...
Reading lineProducingSpecies...
Reading linedata...
read num 1
read sym CO
nlev = 41
nrad = 1
Reading collisionPartner...
Reading collisionPartner...
Reading quadrature...
Reading radiation...
Reading frequencies...
Not using scattering!
                                           
-------------------------------------------
  Model read, parameters:                  
-------------------------------------------
  npoints    = 88846
  nrays      = 2
  nboundary  = 386
  nfreqs     = 31
  n

## Model the medium

Initialize the model by setting up a spectral discretisation, computing the inverse line widths and initializing the level populations with their LTE values.

In [5]:
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

Computing spectral discretisation...
Computing inverse line widths...
Computing LTE level populations...


0

In this example we will work with the LTE level populations and **do not demand** statistical equilibrium.

In [6]:
# Iterate level populations until statistical equilibrium
# model.compute_level_populations (False, 1)

## Make synthetic observations

Now we can make synthetic observations of the model.

In [7]:
fcen = model.lines.lineProducingSpecies[0].linedata.frequency[0]
vpix = 1500   # velocity pixel size [m/s] 
dd   = vpix * (model.parameters.nfreqs()-1)/2 / magritte.CC
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

model.compute_spectral_discretisation (fmin, fmax)
model.compute_image (model.parameters.hnrays()-1)

Computing spectral discretisation...
Computing image...


0

## Plot observations

Plot the resulting channel maps with the Magritte matplotlib back end.

In [8]:
plot.image_mpl(
    model,
    image_nr =  -1,
    zoom     = 1.3,
    npix_x   = 240,
    npix_y   = 240,
    x_unit   = units.au,
    v_unit   = units.km / units.s
)

100%|███████████████████████████████████████████| 31/31 [00:16<00:00,  1.91it/s]


interactive(children=(IntSlider(value=15, description='v', max=30), Output()), _dom_classes=('widget-interact'…

<function magritte.plot.image_mpl.<locals>.<lambda>(v)>

(The plot is only interactive in a live notebook.)

Save the image cube in a fits file.

In [9]:
tools.save_fits(model)

Written file to: /lhome/thomasc/Magritte-examples/Phantom_3D/images/image.fits
